In [ ]:
import pandas as pd
from datetime import datetime

In [28]:
file_path = '../Data/Sterling -- transactions.xlsx'

# Function to import transactions from Excel
def import_transactions(file_path):
    df = pd.read_excel(file_path, header=None)

    i = 0
    transactions = []

    while i < len(df.columns):
        if pd.notna(df.iloc[1, i]):
            # Read security name and ISIN from the second and third lines
            security_name = df.iloc[1, i]
            isin = df.iloc[2, i]

            # Find the start of the transactions table
            transactions_start = df[df.iloc[:, i] == 'Дата'].index[0] + 1

            # Iterate over transactions
            for row in range(transactions_start, len(df)):
                if pd.isna(df.iloc[row, i]):
                    break

                date_str = df.iloc[row, i]
                date = date_str
                # try:
                #     date = datetime.strptime(date_str, '%d/%m/%Y').date()
                # except ValueError:
                #     print(f"Skipping invalid date format: {date_str}")
                #     continue

                price = df.iloc[row, i + 1]
                quantity = df.iloc[row, i + 2]
                dividend = df.iloc[row, i + 3] if not pd.isna(df.iloc[row, i + 3]) else None
                commission = df.iloc[row, i + 4] if not pd.isna(df.iloc[row, i + 4]) else None

                # # Append the transaction details to the list
                transactions.append({
                    'security_name': security_name,
                    'isin': isin,
                    'date': date,
                    'price': price,
                    'quantity': quantity,
                    'dividend': dividend,
                    'commission': commission,
                })

            # Move to the next mini-table by finding the next non-empty second line after an empty column
            i += 1
            while i < len(df.columns) and pd.isna(df.iloc[1, i]):
                i += 1
        else:
            i += 1

    # Convert the list of transactions to a DataFrame
    transactions_df = pd.DataFrame(transactions)
    return transactions_df

import_transactions(file_path)

,security_name,isin,date,price,quantity,dividend,commission
0,M&G Short Dated Corporate Bond,GB0031110397,2010-02-15,24.50,4.082909,None,None
1,M&G Short Dated Corporate Bond,GB0031110397,2010-03-10,24.50,0.408291,None,None
2,M&G Short Dated Corporate Bond,GB0031110397,2010-04-12,24.50,0.408291,None,None
3,M&G Short Dated Corporate Bond,GB0031110397,2010-05-10,24.50,0.408291,None,None
4,M&G Short Dated Corporate Bond,GB0031110397,2010-06-10,24.29,0.411821,None,None
...,...,...,...,...,...,...,...
259,Aberdeen Emerging Markets,GB0033228197,2011-09-09,496.27,0.090705,None,None
260,Aberdeen Emerging Markets,GB0033228197,2011-10-10,460.10,0.097835,None,None
261,Aberdeen Emerging Markets,GB0033228197,2011-11-10,480.40,0.093701,None,None
262,Aberdeen Emerging Markets,GB0033228197,2011-12-09,475.85,0.094597,None,None


I plan to parse my existing Excel for the data to fill the database. This is the data for the transaction for all relevant securities. The data is kept wihin one single spreadsheet. I need to properly find the respective data, which is the date and corresponding info: price, quantity (negative if sale), commission if exists and dividends.

The data is represented as follows:
- Each ticker has its own table. Tables for tickers are positioned side by side
- In the second line I have the name of the security
- In the third line there is ISIN
- Then there are a number of lines with calcs and respective information
- I will need to find the area where the transactions are starting. This is the line where in the first column of the mini-table I have 'Date'
- First column has dates 
- Second column has prices
- Third column: quantity
- Fourth column: dividends
- Fifth column: commission
- Other columns are technical
- The last column of mini-table is the one which is situation before the completely empty column
- The first column of the next mini-table is the one after the completely empty column
- In the beginning of the scan I will need to check whether the security exists in the database based on the security name and ISIN